# Analysis of DIC data from tensile testing

## Import libraries

In [ ]:
import glob

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from ipywidgets import interactive, fixed
from dic_analysis import DeformationMap, scrub_frames

%matplotlib inline 
%load_ext autoreload
%autoreload 2

## Analysis set up
Change the data location and number of files to load here.

In [ ]:
data_folder = "../Data/test 0-1/displacement data/"

Load data from files

In [ ]:
# Load data from files
file_list = glob.glob(f"{data_folder}*")
deformation_maps = [DeformationMap(file_list[file_path]) for file_path in range(0, 20)]

## Plotting strain maps

This cell allows the visualisation of individual strain maps.

In [ ]:
timestep = 10

plt.figure()
plt.imshow(deformation_maps[timestep].f22, vmin=0, vmax=0.4, cmap='viridis', interpolation='none')
plt.colorbar()
plt.title(f"Strain map at timestep {timestep}", pad=10)
plt.show()
#plt.savefig("../results/strain_map_t_{timestep}.png")

This cell is the same but allows scanning through the strain maps over time. The average argument specifies how many
adjacent frames to average - this can smooth the data if necessary.

In [ ]:
figure = plt.figure()
interactive(scrub_frames, measurement=(0, np.size(deformation_maps) - 2, 1), fig=fixed(figure), 
            f_list=fixed(deformation_maps), continuous_update=False)

## Plotting shape change of sample

In [ ]:
timestep = 10

defmap = deformation_maps[timestep]
rho = -defmap.f11 / defmap.f22
# rho is the shape change -deyy/dexx
plt.imshow(rho)
plt.show()

## Plotting sample strain over time

In [ ]:
strain_list = []
for def_map in deformation_maps:
    cropped_map = def_map.f22[10:24, 1:12]
    
    strain_list.append(np.mean(cropped_map))

plt.figure()
plt.plot(strain_list)
plt.xlabel("Time step")
plt.ylabel("Strain")
plt.show()

## Plotting True strain over time

In [ ]:
plt.figure()
plt.plot(np.log(1 + np.array(strain_list)))
plt.xlabel("Time step")
plt.ylabel("True Strain")
plt.show()

## Plotting ?? Over time

In [ ]:
trans_list = []
for def_map in deformation_maps:
    cropped_map = def_map.f11[10:24,1:12]
    
    trans_list.append(np.mean(cropped_map))
    
plt.figure()
plt.plot(trans_list)
plt.plot(strain_list)
plt.xlabel('Data Point')
plt.ylabel('Strain')
plt.show()

## Plotting ??

In [ ]:
strain_ratio = - np.array(trans_list) / np.array(strain_list)
plt.figure()
plt.plot(strain_list[20:], strain_ratio[20:])
plt.show()

## Plotting ??

In [ ]:
Lankford = strain_ratio / (1 - strain_ratio)
plt.figure()
plt.plot(strain_list[100:], Lankford[100:])
plt.show()

## Plotting Measured strain data

In [ ]:
test_names = ["0-1", "0-2", "30-1", "30-2", "45-1", "45-2", "60-1", "60-2", "90-1", "90-2"]

plt.figure()

for name in test_names:
    voltage_data = np.loadtxt(f"../Data/test {name}/voltage data/data_1.csv", delimiter=",", skiprows=2, usecols=(4, 15))
    
    # Cut off data when it begins dropping at the end of the experiment
    data_limit = voltage_data.shape[0]
    for i in range(0, data_limit - 50):
        if voltage_data[i, 1] > voltage_data[i + 50, 1]:
            data_limit = i + 50
            break
            
    # Plot the data
    plt.plot(voltage_data[:data_limit , 0], voltage_data[:data_limit, 1], "x-", label=name)
    plt.xlabel("True Strain")
    plt.ylabel("True Stress (MPa)")
    plt.legend()
plt.show()